In [1]:
from pathlib import Path
import utils.sonar_analysis as iau

# Centralized sonar defaults (inserted by sweep)
from utils.config import SONAR_VIS_DEFAULTS
config = SONAR_VIS_DEFAULTS.copy()
# Backwards-compatible variable names used in older notebooks
RANGE_MIN_M = config['range_min_m']
RANGE_MAX_M = config['range_max_m']
DISPLAY_RANGE_MAX_M = config['display_range_max_m']

 CONFIG VALIDATION WARNING:
   Missing TRACKING_CONFIG keys: {'use_corridor_splitting', 'corridor_widen', 'corridor_both_directions'}


# Sonar Image Analysis

This notebook analyzes sonar imagery for net detection and distance measurement using a multibeam sonar mounted on an ROV at a full-scale fish farm.

## Dataset Information

**Location:** Full-scale fish farm, Norway  
**Fish Cage:** Diameter 50m, 157m circumference, 27.5mm mesh (biofouling present)  
**Biomass:** ~188,000 fish (~3kg average weight)  
**Environment:** Current 0.04-0.2 m/s, Wind 6 m/s, Rain, 14 degrees C air temp

**Sensors Available:**
- Multibeam sonar (Sonoptix Echo), Ping 360 sonar
- DVL (Doppler Velocity Log), IMU, Gyroscope, USBL
- Mono and stereo cameras, Depth, pressure, temperature

## Available Datasets

### Calibration Runs (2024-08-20)
| Timestamp | Type | Description 
|-----------|------|-------------|
| 13-39-34 | Calibration | Stereo camera calibration | 
| 13-40-35 | Calibration | Stereo camera calibration | 
| 13-42-51 | Calibration | Stereo camera calibration | 

### Manual Control Runs (2024-08-20)
| Timestamp | Depth | Description |
|-----------|-------|-------------|
| 13-55-34 | Shallow | Manual control |
| 13-57-42 | Shallow | Manual control |
| 14-16-05 | Deeper | Manual control |
| 14-22-12 | Deeper | Manual control |
| 14-24-35 | Deeper | Manual control |
| 14-31-29 | Shallow | Manual control |

### Net Following - Variable Distance (2024-08-20)
| Timestamp | D0 [m] | D1 [m] | Z [m] | V [m/s] | Q [deg] | Notes |
|-----------|--------|--------|-------|---------|---------|-------|
| **14-57-38** | **2.0** | **1.1** | **2** | **0.2** | **0** | **[RECOMMENDED] Distance change** |
| 15-00-24 | 1.5 | 1.5 | 5 | 0.2 | 0 | Deep |
| **15-14-40** | **1.4** | **1.9** | **5** | **0.1** | **0** | **Deep + slow** |

### Net Following - Consistent Distance (2024-08-20)
| Timestamp | D0 [m] | D1 [m] | Z [m] | V [m/s] | Q [deg] | Category |
|-----------|--------|--------|-------|---------|---------|----------|
| **16-57-46** | **1.0** | **1.5** | **2** | **0.1** | **0** | **Slow [RECOMMENDED]** |
| **17-02-00** | **1.0** | **1.5** | **2** | **0.1** | **0** | **Slow [RECOMMENDED]** |
| **17-08-14** | **0.5** | **1.0** | **2** | **0.1** | **0** | **Close [CHALLENGING]** |
| **17-14-36** | **1.0** | **1.5** | **2** | **0.3** | **0** | **Fast [CHALLENGING]** |
| **17-39-32** | **1.0** | **1.5** | **5** | **0.2** | **0** | **Deep** |
| **17-40-54** | **1.0** | **1.5** | **5** | **0.2** | **0** | **Deep** |
| **18-12-20** | **0.5** | **1.0** | **5** | **0.1** | **0** | **Deep + Slow + Close [CHALLENGING]** |
| **18-47-40** | **1.0** | **1.0** | **2** | **0.2** | **10** | **Angled [CHALLENGING]** |

### Dual-DVL Runs (2024-08-22)
Includes Waterlinked A50 and Nortek Nucleus 1000 measurements.

| Timestamp | D0 [m] | D1 [m] | Z [m] | V [m/s] | Q [deg] | Description |
|-----------|--------|--------|-------|---------|---------|-------------|
| 14-06-43 | 0.5 | 1.0 | 2 | 0.2 | 0 | Standard |
| **14-29-05** | **0.6** | **0.8** | **2** | **0.1** | **0** | **Tight distance [RECOMMENDED]** |
| **14-47-39** | **0.6** | **0.6** | **2** | **0.1** | **0** | **Constant distance [RECOMMENDED]** |

**Legend:**
- **D0:** Initial desired distance to net [m]
- **D1:** Final desired distance to net [m]
- **Z:** Depth [m]
- **V:** Net-relative velocity [m/s]
- **Q:** Heading-angle offset from net [deg]
- **[RECOMMENDED]** Good for initial analysis
- **[CHALLENGING]** Difficult conditions

## Analysis Pipeline
1. Load sonar cone-view images from NPZ
2. Binary conversion (threshold-based)
3. Edge enhancement
4. Net tracking with ellipse fitting
5. Distance measurement (pixels -> meters)
6. Comparison with DVL ground truth

In [2]:
TARGET_BAG = '2024-08-20_17-02-00'  # change this to your desired bag ID
from utils.config import EXPORTS_DIR_DEFAULT, EXPORTS_SUBDIRS
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

# If you keep your raw .bag files or raw data on an external drive, point DATA_DIR there.
# Example (your external disk): /Volumes/LaCie/SOLAQUA/raw_data
DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/raw_data")
print(f"Using DATA_DIR = {DATA_DIR}")

# NOTE: To export data for this specific bag, use:
# python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem {TARGET_BAG} --all

# Find NPZ files and pick the one matching TARGET_BAG
files = iau.get_available_npz_files()
if not files:
    raise FileNotFoundError(f"No NPZ files found in configured exports outputs (looked under {EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('outputs','outputs')})")
matches = [p for p in files if TARGET_BAG in p.name]
if not matches:
    # Help the user by listing available NPZ files
    print(f'No NPZ file matched TARGET_BAG={TARGET_BAG!r}')
    print('Available NPZ files:')
    for i,p in enumerate(files):
        print(f'  {i}: {p.name}')
    raise ValueError(f'No NPZ file contains "{TARGET_BAG}" in its name')

# If multiple matches, choose the most recently modified one
selected = max(matches, key=lambda p: p.stat().st_mtime)
NPZ_FILE_INDEX = files.index(selected)

Using DATA_DIR = /Volumes/LaCie/SOLAQUA/raw_data


## Distance Analysis Over Time

Now let's perform a comprehensive analysis of the red line distance over time. The red line represents the major axis of the detected elongated contour (likely a fishing net), and we'll track how this distance changes throughout the video sequence.

In [3]:
# Import video generation module
import utils.video_generation as sg
import importlib

# Force reload to pick up config changes
importlib.reload(sg)

# Verify the config value is correct
from utils.config import TRACKING_CONFIG as CHECK_CONFIG
print(f"Verified expansion factor: {CHECK_CONFIG['ellipse_expansion_factor']}")

print("GENERATING CONTOUR DETECTION PIPELINE VIDEO")
print("=" * 60)

# Generate contour detection video showing the exact analysis pipeline
try:
    video_path = sg.create_enhanced_contour_detection_video(
        npz_file_index=NPZ_FILE_INDEX,          
        frame_start=1,
        frame_count= 3000,
        frame_step=1,
        output_path=str(Path(EXPORTS_DIR_DEFAULT) / EXPORTS_SUBDIRS.get('videos','videos') / f"contour_detection_{TARGET_BAG}.mp4")
    )
    
except Exception as e:
    print(f"❌ Video generation failed: {e}")
    import traceback
    traceback.print_exc()

Verified expansion factor: 0.5
GENERATING CONTOUR DETECTION PIPELINE VIDEO
=== CONTOUR DETECTION PIPELINE VIDEO (3x3 Grid with NetTracker) ===
Processing 698 frames...
Grid layout (2x3):
  Row 1: Raw | Momentum-Merged | Edges
  Row 2: Search Mask | Best Contour | Distance
Output grid size: 2700x1400
Tracker config:
  expansion: 0.5
  center_alpha: 0.8
  size_alpha: 0.01
  angle_alpha: 0.1
Processing 698 frames...
Grid layout (2x3):
  Row 1: Raw | Momentum-Merged | Edges
  Row 2: Search Mask | Best Contour | Distance
Output grid size: 2700x1400
Tracker config:
  expansion: 0.5
  center_alpha: 0.8
  size_alpha: 0.01
  angle_alpha: 0.1
Processed 50/698 frames
Processed 50/698 frames
Processed 100/698 frames
Processed 100/698 frames
Processed 150/698 frames
Processed 150/698 frames
Processed 200/698 frames
Processed 200/698 frames
Processed 250/698 frames
Processed 250/698 frames
Processed 300/698 frames
Processed 300/698 frames
Processed 350/698 frames
Processed 350/698 frames
Processed 4

In [4]:
# Analyze distance over time using functional API
print("\n" + "=" * 60)
print("ANALYZING DISTANCE OVER TIME")
print("=" * 60)

net_analysis_results = iau.analyze_npz_sequence(
    npz_file_index=NPZ_FILE_INDEX,    
    frame_start=1,
    frame_count=3000,
    frame_step=1,
    save_outputs=True
)


ANALYZING DISTANCE OVER TIME
Analyzing 698 frames from 2024-08-20_13-39-34_data_cones.npz
Bag ID: 2024-08-20_13-39-34
Using NetTracker system with binary processing and ellipse fitting
Analyzing 698 frames from 2024-08-20_13-39-34_data_cones.npz
Bag ID: 2024-08-20_13-39-34
Using NetTracker system with binary processing and ellipse fitting
  50/698 | Status: TRACKED
  50/698 | Status: TRACKED
  100/698 | Status: TRACKED
  100/698 | Status: TRACKED
  150/698 | Status: TRACKED
  150/698 | Status: TRACKED
  200/698 | Status: TRACKED
  200/698 | Status: TRACKED
  250/698 | Status: TRACKED
  250/698 | Status: TRACKED
  300/698 | Status: TRACKED
  300/698 | Status: TRACKED
  350/698 | Status: TRACKED
  350/698 | Status: TRACKED
  400/698 | Status: TRACKED
  400/698 | Status: TRACKED
  450/698 | Status: TRACKED
  450/698 | Status: TRACKED
  500/698 | Status: TRACKED
  500/698 | Status: TRACKED
  550/698 | Status: TRACKED
  550/698 | Status: TRACKED
  600/698 | Status: TRACKED
  600/698 | Stat

In [5]:
net_analysis_results.columns

Index(['frame_index', 'timestamp', 'distance_pixels', 'distance_meters',
       'angle_degrees', 'detection_success', 'tracking_status', 'area'],
      dtype='object')

## Convert to Real-World Distances

Now let's convert the pixel distances to real-world distances using the fact that the entire sonar image represents a 10x10 meter area.

In [6]:
# Auto-detect pixel->meter mapping from the selected NPZ using utils function
from utils.sonar_utils import get_pixel_to_meter_mapping
mapping_info = get_pixel_to_meter_mapping(selected)

# Extract commonly used variables for backwards compatibility
pixels_to_meters_avg = mapping_info['pixels_to_meters_avg']
image_shape = mapping_info['image_shape'] 
sonar_coverage_meters = mapping_info['sonar_coverage_meters']

print(f"Using pixels_to_meters_avg = {pixels_to_meters_avg:.6f} m/px")
print(f"Mapping source: {mapping_info['source']}")

Using pixels_to_meters_avg = 0.016765 m/px
Mapping source: npz_metadata


In [7]:
from utils.sonar_visualization import plot_distance_analysis
plot_distance_analysis(net_analysis_results, "Real-World Distance Analysis")

In [8]:
# COMPARISON: SONAR vs DVL DISTANCE MEASUREMENTS
# =================================================
import utils.distance_measurement as sda
from utils.sonar_visualization import compare_sonar_vs_dvl

# IMPORTANT: Pass the by_bag folder, not just the exports root
# The function expects the folder containing the CSV files
from utils.config import EXPORTS_SUBDIRS
BY_BAG_FOLDER = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag')

# Load all distance data for the target bag
raw_data, distance_measurements = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)

# Display what we loaded
print(f"\nRAW DATA LOADED:")
for key, data in raw_data.items():
    if data is not None:
        print(f"    {key}: {len(data)} records")
    else:
        print(f"    {key}: None")

print(f"\nDISTANCE MEASUREMENTS LOADED:")
for key, info in distance_measurements.items():
    data_len = len(info['data'])
    print(f"    {key}: {data_len} records - {info['description']}")

 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_13-39-34
 1. Loading Navigation Data...
    Loaded 311 navigation records
    NetPitch data available: 311 valid records
 2. Loading Guidance Data...
    Loaded 293 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
 3. Loading DVL Altimeter...
    DVL altimeter file not found
 4. Loading USBL...
    Loaded 21 USBL records
 5. Loading DVL Position...
    Loaded 157 DVL position records
 6. Loading Navigation Position...
    Loaded 153 navigation position records
 7. Loading INS Z Position...
    INS file not found

 LOADING SUMMARY:
    Target bag: 2024-08-20_13-39-34
    Raw data loaded: 2/2
    Distance measurements: 4

RAW DATA LOADED:
    navigation: 311 records
    guidance: 293 records

DISTANCE MEASUREMENTS LOADED:
    USBL_3D: 21 records - 3D acoustic position
    USBL_Depth: 21 records - USBL depth measurement
    DVL_Position: 157 records - 3D DVL position
    Nav_Position: 153 records - 2D n

In [9]:
# Compare sonar vs DVL using the sonar_visualization function
print(f"\n" + "=" * 60)
print("SONAR vs DVL COMPARISON")
print("=" * 60)

fig, comparison_stats = compare_sonar_vs_dvl(
    net_analysis_results,
    raw_data,
    sonar_coverage_m=sonar_coverage_meters,
    sonar_image_size=image_shape[0]
)

if fig:
    fig.show()

print(f"\nComparison Statistics:")
for key, value in comparison_stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")


SONAR vs DVL COMPARISON

Comparison: Sonar=2.373m, DVL=1.796m, Ratio=1.321x
Saved plot: /Volumes/LaCie/SOLAQUA/exports/plots/sonar_vs_dvl_2024-08-20_11-39-38.html
Saved plot: /Volumes/LaCie/SOLAQUA/exports/plots/sonar_vs_dvl_2024-08-20_11-39-38.html
Saved plot: /Volumes/LaCie/SOLAQUA/exports/plots/sonar_vs_dvl_2024-08-20_11-39-38.png
Saved plot: /Volumes/LaCie/SOLAQUA/exports/plots/sonar_vs_dvl_2024-08-20_11-39-38.png



Comparison Statistics:
  sonar_mean_m: 2.373
  dvl_mean_m: 1.796
  scale_ratio: 1.321
  sonar_frames: 698
  dvl_records: 311
  bag_id: 2024-08-20_11-39-38


In [3]:
# CONFIGURATION
FFT_CSV_PATH = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose")
TARGET_FFT_FILE = FFT_CSV_PATH / f"{TARGET_BAG}_relative_pose_fft.csv"

# VIDEO OVERLAY OPTIONS
INCLUDE_TEXT_OVERLAYS = False   # Set to True to show timestamps, frame numbers, distances
INCLUDE_NET_LINES = False       # Set to False to hide all net detection lines

print("GENERATING THREE-SYSTEM VIDEO")
print("=" * 60)
print(f"Target Bag: {TARGET_BAG}")
print(f"Text Overlays: {'ON' if INCLUDE_TEXT_OVERLAYS else 'OFF'}")
print(f"Net Detection Lines: {'ON' if INCLUDE_NET_LINES else 'OFF'}")
print()
print("DEBUG: INCLUDE_NET_LINES =", INCLUDE_NET_LINES)
print()

# Check if FFT data is available
if TARGET_FFT_FILE.exists():
    print(f"✓ FFT data found: {TARGET_FFT_FILE.name}")
    fft_csv_to_use = TARGET_FFT_FILE
else:
    print(f"  FFT data not found: {TARGET_FFT_FILE.name}")
    print(f"  Video will use Sonar + DVL only (2-system)")
    fft_csv_to_use = None

# Import the utility function
from utils.video_generation import generate_three_system_video

try:
    # Generate video with overlay options
    video_path = generate_three_system_video(
        target_bag=TARGET_BAG,
        exports_folder=EXPORTS_FOLDER,
        fft_csv_path=fft_csv_to_use,
        start_idx=1,
        end_idx=3000,
        include_text_overlays=INCLUDE_TEXT_OVERLAYS,  
        include_net_lines=INCLUDE_NET_LINES           
    )
    
    if video_path:
        print(f"\n✓ Video generated successfully: {video_path}")
        import os
        if os.path.exists(video_path):
            size_mb = os.path.getsize(video_path) / (1024**2)
            print(f"  File size: {size_mb:.2f} MB")
    
except FileNotFoundError as e:
    print(f"\n❌ Missing required data files:")
    print(f"  {e}")
    print(f"\nMake sure you've run the sonar analysis cell above with save_outputs=True")
    
except Exception as e:
    print(f"\n❌ Video generation failed: {e}")
    import traceback
    traceback.print_exc()

GENERATING THREE-SYSTEM VIDEO
Target Bag: 2024-08-20_17-02-00
Text Overlays: OFF
Net Detection Lines: OFF

DEBUG: INCLUDE_NET_LINES = False

✓ FFT data found: 2024-08-20_17-02-00_relative_pose_fft.csv
GENERATING THREE-SYSTEM VIDEO
📊 Loading sonar analysis results from saved CSV...
✓ Loaded 944 sonar analysis records from 2024-08-20_17-02-00_analysis.csv
📊 Loading DVL navigation data...
 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_17-02-00
 1. Loading Navigation Data...
    Loaded 497 navigation records
    NetPitch data available: 497 valid records
 2. Loading Guidance Data...
    Loaded 509 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
 3. Loading DVL Altimeter...
    DVL altimeter file not found
 4. Loading USBL...
    Loaded 30 USBL records
 5. Loading DVL Position...
    Loaded 265 DVL position records
 6. Loading Navigation Position...
    Loaded 245 navigation position records
 7. Loading INS Z Position...
    INS file not found

 LOAD